In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchsummary
import sklearn
from sklearn import metrics
from sklearn import naive_bayes

In [2]:
class BasicResidualSEBlock(nn.Module):

    expansion = 1

    def __init__(self, in_channels, out_channels, stride, r=16):
        super().__init__()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1),
            nn.Dropout(),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(out_channels, out_channels * self.expansion, 3, padding=1),
            nn.Dropout(),
            nn.BatchNorm2d(out_channels * self.expansion),
            nn.ReLU(inplace=True)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, 1, stride=stride),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(out_channels * self.expansion, out_channels * self.expansion // r),
            nn.ReLU(inplace=True),
            nn.Linear(out_channels * self.expansion // r, out_channels * self.expansion),
            nn.Sigmoid()
        )

    def forward(self, x):
        shortcut = self.shortcut(x)
        residual = self.residual(x)

        squeeze = self.squeeze(residual)
        squeeze = squeeze.view(squeeze.size(0), -1)
        excitation = self.excitation(squeeze)
        excitation = excitation.view(residual.size(0), residual.size(1), 1, 1)

        x = residual * excitation.expand_as(residual) + shortcut

        return F.relu(x)

In [3]:
class SEResNet(nn.Module):

    def __init__(self, block, block_num, class_num=2):
        super().__init__()

        self.in_channels = 16

        self.pre = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )

        self.stage1 = self._make_stage(block, block_num[0], 16, 1)
        self.stage2 = self._make_stage(block, block_num[1], 32, 2)
        self.stage3 = self._make_stage(block, block_num[2], 64, 2)
        self.stage4 = self._make_stage(block, block_num[3], 128, 2)

        self.linear = nn.Linear(self.in_channels, class_num)

    def forward(self, x):
        x = self.pre(x)

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)

        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(x.size(0), -1)

        x = self.linear(x)

        return x


    def _make_stage(self, block, num, out_channels, stride):

        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion

        while num - 1:
            layers.append(block(self.in_channels, out_channels, 1))
            num -= 1

        return nn.Sequential(*layers)

def seresnets():
    return SEResNet(BasicResidualSEBlock, [3, 4, 6, 3])

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = seresnets()
net.to(device)
torchsummary.summary(net, input_size=(1, 48, 48))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 48, 48]             160
       BatchNorm2d-2           [-1, 16, 48, 48]              32
              ReLU-3           [-1, 16, 48, 48]               0
            Conv2d-4           [-1, 16, 48, 48]           2,320
           Dropout-5           [-1, 16, 48, 48]               0
       BatchNorm2d-6           [-1, 16, 48, 48]              32
              ReLU-7           [-1, 16, 48, 48]               0
            Conv2d-8           [-1, 16, 48, 48]           2,320
           Dropout-9           [-1, 16, 48, 48]               0
      BatchNorm2d-10           [-1, 16, 48, 48]              32
             ReLU-11           [-1, 16, 48, 48]               0
AdaptiveAvgPool2d-12             [-1, 16, 1, 1]               0
           Linear-13                    [-1, 1]              17
             ReLU-14                   